In [1]:
$connect("192.168.0.171")

import platform
print(platform.uname())

uname_result(system='Linux', node='sipeed', release='4.9.118', version='#7 PREEMPT Mon Dec 7 10:27:38 UTC 2020', machine='armv7l', processor='')


In [1]:
'''

## 目前关于 Python3 在 V831 上消耗的内存统计

Python3 运行的结果。

- python3_3.8.5-2_sunxi.ipk

  853 root     5352  T    python3
  
- python3-rpyc (pip install)

  846 root     11844 S    python3 /root/rpyc_test.py

如下依赖库需运行可知。

- python3-pillow_7.2.0-1_sunxi.ipk
- python3-numpy_1.19.2-1_sunxi.ipk

'''

import gc
gc.collect()

import os
def shell(cmd):
    import os
    textlist = os.popen(cmd).readlines()
    for line in textlist:
        print(line)

shell('free -m')

shell('echo 3 > /proc/sys/vm/drop_caches')

shell("ps -w | grep %d | head -n 1" % os.getpid())

# shell("cat /proc/%s/status" % os.getpid()) # put mem


             total       used       free     shared    buffers     cached

Mem:         59756      57752       2004          8       1192      14648

-/+ buffers/cache:      41912      17844

Swap:       262140         28     262112

  805 root     11400 S    python3 /root/rpycs.py



In [1]:
import numpy
shell('free -m')

             total       used       free     shared    buffers     cached

Mem:         59756      57456       2300          8        996       8580

-/+ buffers/cache:      47880      11876

Swap:       262140        260     261880



In [1]:
from PIL import Image
shell('free -m')

             total       used       free     shared    buffers     cached

Mem:         59756      58044       1712          8        408       9168

-/+ buffers/cache:      48468      11288

Swap:       262140        336     261804



In [ ]:
'''

# 1. 一般启动

11068  18%   0% python rpyc_test.py

# 2. 设置 320 * 240 的纯色 Jpeg 图片传输。

17436  29%  68% python rpyc_test.py

# 3. 设置 640 * 480 的纯色 Jpeg 图片传输。

19172  32%  73% python rpyc_test.py

# 4. 最大观察到的情况如下，可能是变量没有被 GC 主动回收。

26620  44%  41% python rpyc_test.py

在 2. 启动图传测试增加 5M ，加载 from PIL import Image 要 2M
存在 image 原图 + 压缩缓冲 两个变量，各 320*240*3 *2 理论约 500K。

剩下 2.5M 被系统消耗的不明不白。

到 3. 时 640*480*3*2 约 897k 则下面增加了 1M 多的情况，比较符合预期。

而 4. 情况可能是 3 和 4 阶段的变量同时存在。

'''

In [1]:
#! /usr/bin/python
import ctypes
import sys

size = int(200)
class MemoryTest(ctypes.Structure):
    _fields_ = [  ('chars' , ctypes.c_char*size * 1024*1024 ) ]

try:
    test = MemoryTest()
    print('success => {0:>4}MB was allocated'.format(size) )
except:
    print('failure => {0:>4}MB can not be allocated'.format(size) )
    
shell('free -m')

2020-12-22 18:30:27,113 - rpyc_ikernel - ERROR - name 'shell' is not defined

========= Remote Traceback (1) =========
Traceback (most recent call last):
  File "/usr/lib/python3.8/site-packages/rpyc/core/protocol.py", line 320, in _dispatch_request
    res = self._HANDLERS[handler](self, *args)
  File "/usr/lib/python3.8/site-packages/rpyc/core/protocol.py", line 593, in _handle_call
    return obj(*args, **dict(kwargs))
  File "/usr/lib/python3.8/site-packages/rpyc/core/service.py", line 152, in execute
    execute(text, self.namespace)
  File "<string>", line 15, in <module>
NameError: name 'shell' is not defined



success =>  200MB was allocated
